<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/how_to_count_token.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In this notebook, we'll see how to use Solar tokenizer to count the number of tokens in a message.

In [ ]:
# @title Install requirements
!pip install -q openai tokenizers
!pip install -q python-dotenv

In [1]:
# @title set API key
import os
import getpass
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata
    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")


In [6]:
# @title Set Solar client
from openai import OpenAI
from tokenizers import Tokenizer

client = OpenAI(api_key=UPSTAGE_API_KEY, base_url="https://api.upstage.ai/v1/solar")
tokenizer = Tokenizer.from_pretrained("upstage/solar-1-mini-tokenizer")

In [3]:
# @title Define messages
messages = [
    {"role": "system", "content": "Your name is Solar. You should answer politely."},
    {
        "role": "user",
        "content": "Hi!",
    },
    {
        "role": "assistant",
        "content": "How can I help you?",
    },
    {
        "role": "user",
        "content": "his tutorial demonstrates how to combine the semantic search results from a $vectorSearch query with full-text search results from a $search query by using reciprocal rank fusion. Reciprocal rank fusion is a way to combine results from different types of searches into a single result. This tutorial takes you through the following steps: Create an Atlas Vector Search index on the plot_embeddings field in the sample_mflix.embedded_movies collection.Create an Atlas Search index on the title field in the sample_mflix.embedded_movies collection. Combine and run a $vectorSearch query against the plot_embeddings field in the sample_mflix.embedded_movies collection with a $search query against the title field by using reciprocal rank fusion.PrerequisitesBefore you begin, ensure that your Atlas cluster meets the requirements described in the Prerequisites. NOTE Ensure that your Atlas cluster has enough memory to store both Atlas Search and Atlas Vector Search indexes and run performant queries. Create the Atlas Vector Search and Atlas Search Indexes This section demonstrates how to create the following indexes on the fields in the sample_mflix.embedded_movies collection: An Atlas Vector Search index on the plot_.",
    },
]

In [7]:
# @title count messages tokens using tokenizer
def count_tokens_from_messages(tokenizer, messages):
    tokens_per_message = 5  # <|im_start|>{role}\n{message}<|im_end|>
    tokens_prefix = 1  # <|startoftext|>
    tokens_suffix = 3  # <|im_start|>assistant\n

    num_tokens = 0
    num_tokens += 1
    for m in messages:
        num_tokens += tokens_per_message
        for _, v in m.items():
            num_tokens += len(tokenizer.encode(v, add_special_tokens=False))

    num_tokens += tokens_suffix
    return num_tokens

In [9]:
# @title Define messages
messages = [{"role": "system", "content": "Hello World!"}]

In [10]:
print(
    count_tokens_from_messages(tokenizer, messages),
    "prompt tokens counted by the Solar tokenizer",
)

13 prompt tokens counted by the Solar tokenizer


In [89]:
# @title Count tokens using solar API
response = client.chat.completions.create(
    model="solar-1-mini-chat", messages=messages, temperature=0
)
print(f"{response.usage.prompt_tokens} prompt tokens counted by the Solar.")
print()

318 prompt tokens counted by the Solar.

